In [1]:
%matplotlib inline

import torch
import pylab as plt
from munch import munchify
from torch.nn import functional as F
from utils.cores import preview_cores, rf2bm_wrapper
from torch.utils.data import TensorDataset, DataLoader

from utils import *
from tqdm import tqdm
from preprocessing.s02_create_dataset import load_cores_h5py

In [2]:
args = {}
args['config'] = '../yamls/coteaching_local_inference.yml'
args['exp_suffix'] = '_sd150_bs2048_lr5e-4_no_norm_ep100_ap1_elr'
args['backbone'] = 'inception'
args['gpus_id'] = [0,]

with open(args['config']) as f:
    opt = yaml.load(f, Loader)
opt.update(args)
opt = munchify(opt)
opt = setup_directories(opt)

num_workers = 12
device = torch.device(f'cuda:{opt.gpus_id[0]}' if torch.cuda.is_available() else 'cpu')

In [3]:
net = construct_network(device, opt)()
net.load_state_dict(torch.load(f'{opt.project_root}/{opt.paths.checkpoint_dir}/{opt.test.which_iter}_coreN_1.pth'))

<All keys matched successfully>

In [4]:
with open('../metadata/matched_tmi_cores_idx.pkl', 'rb') as fp:
    core_indices = pickle.load(fp)

In [5]:
def predict_and_visualize(patient_id):
    _cores = load_cores_h5py(patient_id, core_indices[state][patient_id])

    inputs = []
    for core in _cores:
        if (core.roi[0] == 1).sum() == 0:
            print(core.core_id)
            continue
        inputs.append(core.rf[:, core.roi[0] == 1].T[:, np.newaxis])
    
    if len(inputs) == 0:
        print(patient_id)
        return
    
    dataset = TensorDataset(torch.tensor(np.concatenate(inputs, axis=0), dtype=torch.float32))
    dataloader = DataLoader(dataset, shuffle=False, num_workers=num_workers, batch_size=opt.test.batch_size)

    outputs = []
    net.eval()

    with torch.no_grad():
        with tqdm(dataloader, unit="batch") as t_epoch:
            for i, (data, ) in enumerate(t_epoch):
                output = net(data.cuda())
                outputs.append(F.softmax(output, dim=1).cpu().detach().numpy())

    outputs = np.concatenate(outputs)

    current_idx = 0
    for i, core in enumerate(_cores):
        heatmap = np.zeros_like(core.roi, dtype='float32')
        core_len = int(core.roi.sum())
        heatmap[:, core.roi[0] == 1] = outputs[current_idx: current_idx + core_len, 1]
        core.heatmap = heatmap
        current_idx += core_len

    _cores = [rf2bm_wrapper(core) for core in _cores]
    
    figure_filename = '/'.join((figure_dir, f'Patient{_cores[0].patient_id}.png'))
    fig = preview_cores(_cores)
    fig.savefig(figure_filename, bbox_inches='tight')
    plt.close('all')

In [6]:
for state in ['test', 'val']:
    figure_dir = '/'.join((opt.project_root, opt.paths.result_dir.replace('results', 'figures') + '/' + state))
    os.makedirs(figure_dir, exist_ok=True)
    patient_ids = core_indices[state].keys()
    patient_ids

    with tqdm(patient_ids, unit="patient") as t_patient:
        for i, patient_id in enumerate(t_patient):
            predict_and_visualize(patient_id)

100%|██████████| 7/7 [00:00<00:00,  8.61batch/s]
/home/minh/PycharmProjects/prostate_cancer_classification_v1/utils/cores.py:224: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
  0%|          | 0/8 [00:00<?, ?batch/s]

65



  5%|▍         | 1/21 [00:00<00:14,  1.40batch/s]


100%|██████████| 29/29 [30:53<00:00, 63.92s/patient] 


In [7]:
import gc
gc.collect()
torch.cuda.empty_cache()